# Exercices
 
- Les "fake news" sont des informations erronées ou des désinformations diffusées dans le pays par le bouche-à-oreille et, plus récemment, par les communications numériques tels que les messages de l'application "What's app", les messages des médias sociaux, etc.

- Les fake News se propagent plus rapidement que les vraies News et créent des problèmes et des craintes au sein des groupes et de la société.

- Nous allons nous attaquer à ces problèmes en utilisant des techniques NLP, l'objectif est de prédire (classer)  si un message/texte donné est **un vrai ou un faux message**.
 
L'objectif est de comparer différentes approches de représentation de textes :
- codage de textes simple tf.idf, simple comptage
- codage wordembeddings
- codage par des transformers

et différentes approches de de classification 
- méthode classique (tRegression logistoc, un KNN, ...), bibliothèques : pytorch ou SKLEARN
- méthode basée sur les tranfomers
- - utilisation d'un pipeline (tout prêt)
  - utitisation d'un modèle avec votre propre finetuning

sur deux dataset différents un en francais le second en anglais.


### **Le dataset FakeNews**
- Ces données se composent de trois colonnes : Titre, Texte,  Label
- Le texte est une déclaration ou un message concernant un événement ou une situation particulière.
- La caractéristique label indique si le texte donné est faux ou réel.
- Comme il n'y a que deux classes (Fake ou Réel) , ce problème relève de la classification binaire.
 

## Exercice 1 : Classification classique avec différentes représentations


### Dataset en anglais

In [2]:
#import pandas library
import pandas as pd

#read the dataset with name "data_fake_news" and store it in a variable df
df = pd.read_csv("data_fake_news_en.csv")

#print top 5 
df.head(5)

,title,text,label
0,Three more states refuse Trump commission's vo...,"WASHINGTON (Reuters) - Maryland, Delaware and ...",1
1,"Trump Crosses The Line, Attacks Civil Rights ...",Georgia Congressman John Lewis is one of Ameri...,0
2,IS TRUMP A RACIST? Famous Italian-American “Ge...,Robert Davi gives a great answer to Neil Cavut...,0
3,Ted Cruz Gets His Unethical A** Handed To Him...,Seth Meyers destroyed Republican presidential ...,0
4,Putin says Trump hampered from delivering elec...,"SOCHI, Russia (Reuters) - Russian President Vl...",1


In [3]:
# Vérifier les noms des colonnes
print(df.columns)


Index(['title', 'text', 'label'], dtype='object')


### Exemple de split des données, mais vous pourrez utiliser n'importe quel moyen (bibliothèque)

In [4]:
#import train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.text, 
    df.label, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=42,
    stratify=df.label
)


In [5]:
#print the shapes of X_train and X_test
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (35918,)
Shape of X_test: (8980,)


### **Question 1** :

- Utiliser une représentation simple (CountVectorizer avec unigrams). (on supprime les mots vides, ...)
- Utiliser un classifier de votre choix (par exemple **RandomForest**.
- Afficher le rapport de classification 


**A TITRE INFICATIF voici le schéma que vous pourrez suivre**</br>
Attention, l'exemple ci-dessous se base sur la biliothèque sklearn (ce n'est qu'un exemple);, vous pourrez utiliser n'importe quelle biliothèque.

In [11]:
# Importer les bibliothèques nécessaires
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# 0. Choisir un classifieur, ici RandomForest
clf = RandomForestClassifier(random_state=42)

# 1. Transformer le texte en vecteurs de comptage (CountVectorizer)
cv = CountVectorizer(
    ngram_range=(1, 1),
    stop_words='english',
    max_features=5000,
    min_df=2,
    strip_accents='unicode',
    lowercase=True
)

# Transformer les données d'entraînement et de test
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

clf.fit(X_train_cv, y_train)



RandomForestClassifier(random_state=42)

In [12]:
# 3. Obtenir les prédictions pour X_test et les stocker dans y_pred
y_pred = clf.predict(X_test_cv)

# 4. Afficher le rapport de classification
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4696
           1       1.00      1.00      1.00      4284

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



### **Question 2** 

- Utiliser une représentation combinant unigrams et bi-grammes `CountVectorizer`
- Utiliser un classifier de votre choix 
- Afficher le rapport de classification 


In [13]:
from sklearn.svm import LinearSVC

# définir le modèle
clf = RandomForestClassifier(n_estimators=100, random_state=42)

#1. create a  Transform text to countvectors 

cv_bigram = CountVectorizer(ngram_range=(1, 2),stop_words='english',max_features=5000,min_df=2,strip_accents='unicode',lowercase=True
)

#2. fit_ transform X_train (utiliser fit_transform de sklearn)

X_train_cv_bigram = cv_bigram.fit_transform(X_train)
X_test_cv_bigram = cv_bigram.transform(X_test)

#3. get the predictions for X_test and store it in y_pred

clf.fit(X_train_cv_bigram, y_train)
y_pred = clf.predict(X_test_cv_bigram)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4696
           1       1.00      1.00      1.00      4284

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



### **Question 3** 

- Utiliser **TF-IDF vectorizer**  avec unigrams.
- Utiliser un classifier de votre choix 
- Afficher le rapport de classification 


In [14]:
#import TfidfVectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# définir le modèle
clf = LogisticRegression(random_state=42)

#1. Vectorise the Text 
tfidf = TfidfVectorizer(ngram_range=(1, 1),stop_words='english',max_features=5000,min_df=2,strip_accents='unicode',lowercase=True)

#2. fit (transform) with X_train 
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

#3. get the predictions for X_test and store it in y_pred
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)

#4. print the classfication report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4696
           1       0.98      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



### **Question 4** :

- Représentation d'un texte avec des embeddings (word2vec ou BERT, ...).
- Utiliser un classifier de votre choix (par exemple **RandomForest**.)
- Afficher le rapport de classification 



In [6]:
# Import des librairies nécessaires
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk


# Tokenisation des textes pour Word2Vec
X_train_tokens = [text.split() for text in X_train]
X_test_tokens = [text.split() for text in X_test]

# Entraîner un modèle Word2Vec sur les données d'entraînement
word2vec_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Fonction pour convertir un texte en un vecteur en moyennant les embeddings des mots
def text_to_embedding(tokens, word2vec_model):
    embeddings = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)
#1. Transform text to tfidfvectorizer

X_train_embeddings = np.array([text_to_embedding(tokens, word2vec_model) for tokens in X_train_tokens])
X_test_embeddings = np.array([text_to_embedding(tokens, word2vec_model) for tokens in X_test_tokens])

clf = RandomForestClassifier(random_state=42)
#2. fit with X_train and y_train

clf.fit(X_train_embeddings, y_train)
#3. get the predictions for X_test and store it in y_pred

y_pred = clf.predict(X_test_embeddings)
#4. print the classfication report
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

Rapport de classification :
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4696
           1       0.98      0.98      0.98      4284

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



## 1.2 Refaire le travail sur le Dataset en francais

In [1]:
### Dataset en francais 

In [2]:
# Importer la bibliothèque pandas
import pandas as pd

# Lire le dataset avec le nom "data_fake_news_fr" et le stocker dans une variable df
df = pd.read_csv("data_fake_news_fr.csv")

# Afficher les 5 premières lignes
df.head(5)


,title,description,text,fake
0,Les chefs de service hospitaliers en appellent...,Jugeant très insuffisante la réponse du gouver...,Les chefs de service hospitaliers en appellent...,0
1,L'origine des comportements alimentaires ident...,D'après une étude publiée dans la revue,L'origine des comportements alimentaires ident...,0
2,Microsoft alerte sur de nouvelles techniques d...,Le géant de l'informatique Microsoft vient de ...,Microsoft alerte sur de nouvelles techniques d...,0
3,L'Europe se dote d'une infrastructure de téléc...,Pour garantir la sécurité de la libre circulat...,L'Europe se dote d'une infrastructure de téléc...,0
4,Science décalée : la créature la plus rapide s...,Le guépard est l'animal le plus rapide... sur ...,Science décalée : la créature la plus rapide s...,0


In [3]:
# Vérifier les noms des colonnes
print(df.columns)

Index(['title', 'description', 'text', 'fake'], dtype='object')


In [4]:
# Importer train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.text, 
    df.fake,  # Utiliser la colonne 'fake' pour les labels
    test_size=0.2,  # 20% des échantillons iront dans le dataset de test
    random_state=42,
    stratify=df.fake  # Assurer une stratification basée sur la colonne 'fake'
)

# Afficher les formes de X_train et X_test
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (1945,)
Shape of X_test: (487,)


In [ ]:
### QUESTION 1

In [7]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

# Télécharger la liste des mots vides en français si ce n'est pas déjà fait
nltk.download('stopwords')

# Obtenir la liste des mots vides en français
stop_words_fr = stopwords.words('french')

# Choisir le classifieur, ici RandomForest
clf = RandomForestClassifier(random_state=42)

# Transformer le texte en vecteurs de comptage (CountVectorizer)
cv = CountVectorizer(
    ngram_range=(1, 1),
    stop_words=stop_words_fr,  # Utiliser la liste de mots vides en français
    max_features=5000,
    min_df=2,
    strip_accents='unicode',
    lowercase=True
)

# Transformer les données d'entraînement et de test
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

# Entraîner le modèle avec les données d'entraînement
clf.fit(X_train_cv, y_train)

# 3. Obtenir les prédictions pour X_test et les stocker dans y_pred
y_pred = clf.predict(X_test_cv)

# 4. Afficher le rapport de classification
print(classification_report(y_test, y_pred))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\simon\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['etaient', 'etais', 'etait', 'etant', 'etante', 'etantes', 'etants', 'ete', 'etee', 'etees', 'etes', 'etiez', 'etions', 'eumes', 'eutes', 'fumes', 'futes', 'meme'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.96      0.96      0.96       273
           1       0.95      0.95      0.95       214

    accuracy                           0.96       487
   macro avg       0.96      0.96      0.96       487
weighted avg       0.96      0.96      0.96       487



In [ ]:
### QUESTION 2

In [9]:
# Importer les bibliothèques nécessaires
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

# Télécharger les stopwords en français de nltk
nltk.download('stopwords')
french_stopwords = stopwords.words('french')

# Définir le modèle (ici, RandomForest)
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 1. Créer un CountVectorizer pour transformer le texte en vecteurs de comptage (unigrams et bigrams)
cv_bigram = CountVectorizer(
    ngram_range=(1, 2),  # Utiliser des unigrams et bigrams
    stop_words=french_stopwords,  # Utiliser les mots vides en français
    max_features=5000,  # Limiter à 5000 mots les plus fréquents
    min_df=2,  # Ignorer les mots qui apparaissent dans moins de 2 documents
    strip_accents='unicode',  # Supprimer les accents
    lowercase=True  # Convertir tout en minuscules
)

# 2. Appliquer fit_transform sur X_train
X_train_cv_bigram = cv_bigram.fit_transform(X_train)
X_test_cv_bigram = cv_bigram.transform(X_test)

# 3. Entraîner le modèle avec les données d'entraînement
clf.fit(X_train_cv_bigram, y_train)

# 4. Obtenir les prédictions pour X_test et les stocker dans y_pred
y_pred = clf.predict(X_test_cv_bigram)

# 5. Afficher le rapport de classification
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\simon\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['etaient', 'etais', 'etait', 'etant', 'etante', 'etantes', 'etants', 'ete', 'etee', 'etees', 'etes', 'etiez', 'etions', 'eumes', 'eutes', 'fumes', 'futes', 'meme'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.97      0.95      0.96       273
           1       0.94      0.97      0.95       214

    accuracy                           0.96       487
   macro avg       0.96      0.96      0.96       487
weighted avg       0.96      0.96      0.96       487



In [ ]:
### QUESTION 3

In [11]:
# Importer les bibliothèques nécessaires
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

nltk.download('stopwords')
french_stopwords = stopwords.words('french')

clf = LogisticRegression(random_state=42)

# 1. Vectoriser le texte avec TF-IDF (uniquement des unigrams)
tfidf = TfidfVectorizer(
    ngram_range=(1, 1),  
    stop_words=french_stopwords,  
    max_features=5000, 
    min_df=2,  
    strip_accents='unicode',  
    lowercase=True  
)

# 2. Appliquer fit_transform sur X_train
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# 3. Entraîner le modèle avec les données d'entraînement
clf.fit(X_train_tfidf, y_train)

# 4. Obtenir les prédictions pour X_test et les stocker dans y_pred
y_pred = clf.predict(X_test_tfidf)

# 5. Afficher le rapport de classification
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\simon\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['etaient', 'etais', 'etait', 'etant', 'etante', 'etantes', 'etants', 'ete', 'etee', 'etees', 'etes', 'etiez', 'etions', 'eumes', 'eutes', 'fumes', 'futes', 'meme'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.95      0.95      0.95       273
           1       0.93      0.93      0.93       214

    accuracy                           0.94       487
   macro avg       0.94      0.94      0.94       487
weighted avg       0.94      0.94      0.94       487



In [ ]:
### QUESTION 4

In [13]:
# Importer les librairies nécessaires
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt_tab')
nltk.download('punkt')  

# Tokenisation des textes pour Word2Vec
X_train_tokens = [word_tokenize(text.lower()) for text in X_train]  # Convertir en minuscules et tokeniser
X_test_tokens = [word_tokenize(text.lower()) for text in X_test]    # Convertir en minuscules et tokeniser

# Entraîner un modèle Word2Vec sur les données d'entraînement
word2vec_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Fonction pour convertir un texte en un vecteur en moyennant les embeddings des mots
def text_to_embedding(tokens, word2vec_model):
    embeddings = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# 1. Convertir les textes en embeddings
X_train_embeddings = np.array([text_to_embedding(tokens, word2vec_model) for tokens in X_train_tokens])
X_test_embeddings = np.array([text_to_embedding(tokens, word2vec_model) for tokens in X_test_tokens])

# Définir le classificateur RandomForest
clf = RandomForestClassifier(random_state=42)

# 2. Entraîner le modèle avec X_train_embeddings et y_train
clf.fit(X_train_embeddings, y_train)

# 3. Obtenir les prédictions pour X_test et les stocker dans y_pred
y_pred = clf.predict(X_test_embeddings)

# 4. Afficher le rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Rapport de classification :
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       273
           1       0.87      0.86      0.87       214

    accuracy                           0.88       487
   macro avg       0.88      0.88      0.88       487
weighted avg       0.88      0.88      0.88       487

